In [4]:
!pip install librosa

     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     --------------- ------------------------ 0.4/1.0 MB 12.2 MB/s eta 0:00:01
     ---------------------------------------  1.0/1.0 MB 12.7 MB/s eta 0:00:01
     ---------------------------------------- 1.0/1.0 MB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/253.7 kB ? eta -:--:--
   --------------------------------------- 253.7/253.7 kB 16.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/222.8 kB ? eta -:--:--
   --------------------------------------- 222.8/222.8 kB 13.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------- ----------------------------- 0.7/2.6 MB 20.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.4/2.6 MB 17.4 MB/s eta 0:00:01
   --------------------------------- ------ 2.2/2.6 MB 19.8 MB/s eta 0:00:01
   ---------------------------------------  2.6/2.6 MB 18.4 MB/s eta 0:00:01
   ----------

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df_4 = pd.read_csv("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/most_common/4차년도_most_common (1).csv")
df_5 = pd.read_csv("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/most_common/5차년도_most_common.csv")
df_5_2 = pd.read_csv("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/most_common/5차년도_2차_most_common.csv")

In [16]:
import os
from collections import Counterimport numpy as np
import pandas as pd
import librosa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tqdm.auto import tqdm

In [41]:
# Drive 경로
drive_path = "C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋"

# dataset 경로
dataset_path_4 = f"{drive_path}/4차년도/"
dataset_path_5 = f"{drive_path}/5차년도/5차_wav/"
dataset_path_5_2 = f"{drive_path}/5차년도_2차/"

In [65]:
def extract_features(df):
    mfcc_len_list=[]
    mfcc_data = []
    for name in tqdm(df['wav_id']):
        path = None
        path = f"{dataset_path_4}{name}.wav"
        if(os.path.isfile(path)):
            y, sr = librosa.load(path, sr=48000)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01), n_fft=int(sr*0.02))
            mfcc_data.append(mfccs.T)
            mfcc_len_list.append(len(mfccs.T))
        else:
            # 빈 배열 반환
            print(f"Warning: File not found for {name}")
            #return np.array([])
    return mfcc_data, mfcc_len_list

In [67]:
mfcc_data_4, mfcc_len_list_4 = extract_features(df_4)

  0%|          | 0/14606 [00:00<?, ?it/s]

In [68]:
#ndarray로 변환
df_4_features_np = np.array(mfcc_data_4, dtype=object)

In [69]:
#5,5_2추출 함수
def extract_features_re(df,data_path):
    mfcc_len_list=[]
    mfcc_data = []
    for name in tqdm(df['wav_id']):
        path = None
        path = f"{data_path}{name}.wav"
        if(os.path.isfile(path)):
            y, sr = librosa.load(path, sr=48000)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01), n_fft=int(sr*0.02))
            mfcc_data.append(mfccs.T)
            mfcc_len_list.append(len(mfccs.T))
        else:
            # 빈 배열 반환
            print(f"Warning: File not found for {name}")
            #return np.array([])

    return mfcc_data, mfcc_len_list

In [70]:
#5차 오디오 특성 추출
mfcc_data_5, mfcc_len_list_5=extract_features_re(df_5,dataset_path_5)

  0%|          | 0/10011 [00:00<?, ?it/s]

In [71]:
#5차 2차 오디오 특성추출
mfcc_data_5_2, mfcc_len_list_5_2 = extract_features_re(df_5_2,dataset_path_5_2)

  0%|          | 0/19374 [00:00<?, ?it/s]

In [72]:
#ndarray변환
mfcc_data_5 = np.array(mfcc_data_5, dtype=object)
mfcc_data_5_2 = np.array(mfcc_data_5_2, dtype=object)

In [74]:
#npy로 저장
np.save("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/data_5_features", mfcc_data_5)
np.save("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/data_4_features_np", df_4_features_np)
np.save("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/data_5_2_features_np", mfcc_data_5_2)

In [75]:
max(mfcc_len_list_5)

2505

In [76]:
max(mfcc_len_list_5_2)

5897

In [77]:
max(mfcc_len_list_4)

4412

In [78]:
#ndarray 병합
merged_array = np.concatenate((df_4_features_np, mfcc_data_5,mfcc_data_5_2), axis=0)

In [79]:
#병합한 ndarray 저장
np.save("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/total_features", merged_array)

In [81]:
#병합한 ndarray 불러오기
total_features = np.load("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/total_features.npy",allow_pickle=True)

In [82]:
#패딩. 위의 list에서 최댓값이 5897 이었으므로 이를 기준으로 한다
df_features_padded = pad_sequences(total_features,maxlen=5897, dtype='float32')

In [89]:
#총 특성 데이터 43975개
len(df_features_padded)

43975

In [95]:
#특성데이터 패딩한거 저장
np.save("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/total_features_padded", df_features_padded)

### emotion 레이블 추출

In [85]:
#4차시의 missing 데이터 제거
missing_wav_id = ["5e2979c25807b852d9e018d5","5e298b9f5807b852d9e01a0f","5e298bc45807b852d9e01a10","5e298bdc5807b852d9e01a11","5e298c085807b852d9e01a12","5e2ad4145807b852d9e020d9","5e2ad43e5807b852d9e020dc","5e2998b85807b852d9e01b02","5e3161c65807b852d9e032af","5e31622f5807b852d9e032ba","5e32924e5807b852d9e03894","5e3292655807b852d9e03896","5e3292825807b852d9e0389a","5e315dca5807b852d9e03275","5e33a9d35807b852d9e050f4","5e33638b5807b852d9e04aeb"]
# csv에만 있는데이터를 제거한다
for i in missing_wav_id:
    df_4.drop(df_4[(df_4['wav_id']==i)].index, inplace=True)

In [86]:
#df_5와 df_5_2는 상관없으니까 그냥 병합
y_total = pd.concat([df_4["emotion"], df_5["emotion"],df_5_2["emotion"]])

In [93]:
#다 합친 전체 데이터 프레임 저장
df_total = pd.concat([df_4,df_5,df_5_2])
df_total.to_csv("df_total.csv")

In [90]:
#레이블 길이가 특성 데이터 길이와 같은것을 확인
len(y_total)

43975

In [91]:
#레이블 원핫인코딩
y_one_hot = pd.get_dummies(y_total).values

In [94]:
#원핫 인코딩 레이블 저장
np.save("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/total_labels", y_one_hot)

In [98]:
#레이블 원핫인코딩 후 저장하기
y_one_hot_4 = pd.get_dummies(df_4["emotion"]).values
y_one_hot_5 = pd.get_dummies(df_5["emotion"]).values
y_one_hot_5_2 = pd.get_dummies(df_5_2["emotion"]).values

In [99]:
np.save("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/df_4_labels", y_one_hot_4)
np.save("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/df_5_labels", y_one_hot_5)
np.save("C:/Users/lockd/data/BD/감정 분류를 위한 대화 음성 데이터셋/df_5_2_labels", y_one_hot_5_2)